### Install Dependencies
Make sure you have the required dependencies installed. You can do this by running the following command in your terminal:

In [2]:
%pip install --only-binary=:all: tiktoken
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.4 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.5 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Install required libraries
%pip install -qU \
    langchain==0.3.* \
    langchain_openai==0.3.* \
    langchain_community \
    unstructured[md]==0.17.* \
    langgraph==0.4.* \
    websockets==15.0.*

Note: you may need to restart the kernel to use updated packages.


In [28]:
from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
import requests
import os

from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
if (openai_api_key):
    print("------------------- Loaded Key -------------------")
# print(openai_api_key)  # optional: check it loaded correctly

# Load the OpenAPI specification from the running ticketing system
root = "http://localhost:3000"
api_spec_url = f"{root}/api/docs/openapi.json"
print(api_spec_url)

# Download and parse the OpenAPI spec
response = requests.get(api_spec_url)
data = response.json()
data['servers'] = [{'url': root}]
openapi_spec = reduce_openapi_spec(data, dereference=False)

### Try to get it to categorize all tickets
requests_wrapper = RequestsWrapper()
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

import requests

agent = planner.create_openapi_agent(
    api_spec=openapi_spec,
    requests_wrapper=requests_wrapper,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    handle_parsing_errors=True,
    allow_operations=['GET', 'POST', 'PUT', 'PATCH', 'DELETE']
)


response = requests.get(root+ "/api/tickets")
tickets = response.json()

for t in tickets:
    ticket_id = t["id"]
    ticket_cat = t["category"]
    print("Ticket:", ticket_id)
import json
import websockets

WS_URL = "ws://localhost:3000/ws"

async def post_test_ticket():
    async with websockets.connect(WS_URL) as websocket:
        print("WebSocket connected. Posting test ticket...")

        # Generate ticket JSON using your agent (returns a Python dict)
        ticket_json = agent.invoke("""
        Create a new support ticket for the "Bean Machine" (an automated robot arm for custom coffee drinks). 
        
        Invent the situation yourself. Provide ONLY these four fields:
        - customer_name
        - customer_email
        - subject
        - description
        
        Do NOT include id, created_at, category, priority, or serial_number.
        
        Return a JSON object in this exact format and POST it to the Ticket endpoint:
        
        POST /api/tickets
        Content-Type: application/json
        
        {
          "customer_name": "<name>",
          "customer_email": "<email>",
          "subject": "<short issue summary>",
          "description": "<1-2 sentence problem description>"
        }
        """.strip())



        # Convert dict to JSON string
        ticket_str = json.dumps(ticket_json)

        # Send over WebSocket
        await websocket.send(ticket_str)
        print("Ticket sent successfully!")

# Run in Jupyter or async context
import asyncio
await post_test_ticket()

------------------- Loaded Key -------------------
http://localhost:3000/api/docs/openapi.json
Ticket: 1a2b3c4d-0001-0000-0000-000000000001
Ticket: 1a2b3c4d-0002-0000-0000-000000000002
Ticket: 1a2b3c4d-0003-0000-0000-000000000003
Ticket: 1a2b3c4d-0004-0000-0000-000000000004
Ticket: 843628c5-80fe-488b-afd1-12877ad0f29c
Ticket: 4cf79b5d-3512-4da7-bc3a-1eb5040733d6
Ticket: 61f44ec5-b77e-4069-8906-5cd09023d077
Ticket: ab4cdd2f-6cad-4e3a-af5e-4df6a272c15d
Ticket: 2354ec56-c331-4d3a-8c0a-29ead7bc97c3
Ticket: edd8e4e2-9a1b-49cd-9b31-4526f34fc9d4
Ticket: 86dd6439-19f7-4a70-8f79-3727453e1888
Ticket: b2e79e17-0faf-4092-b382-20d9f82c5a3d
Ticket: e9998a6c-cb97-4897-a9d4-db7bf34ad9db
Ticket: cdb9c751-1660-4a89-b725-e39900434113
Ticket: ed2f1da8-5dc3-453b-864c-55c1f35732ff
Ticket: c7b03e37-b692-4c4c-be01-43720bda1b78
Ticket: 8a6ec151-b0d7-428e-8527-27d5024285dd
Ticket: fd00549c-3125-4cba-9c46-efc1844a2201
Ticket: d1f411c4-e0bc-4545-88c9-44b3a7031dbe
Ticket: 49991205-feaf-48c8-8ce3-bfbeefaa58bf
WebSo

## Basic RAG System
The follow cell sets up a basic Retrieval-Augmented Generation (RAG) retriever for the support information. This allows the agent to access relevant support documents when answering user queries, enhancing its ability to provide accurate and helpful responses. It does this by:

1. Loading the support documents from a specified directory.
2. Creating a vector store to index the documents.
3. Demonstrating how to use the retriever to get relevant information based on a user query.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

# Initialize the embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Load documents from a directory
loader = DirectoryLoader("./support-info")
docs = loader.load()

# Initialize the vector store and add documents to it
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(docs)

# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Use the retriever to find the most relevant documents for a given query
query = "Machine won't start."
relevant_docs = retriever.get_relevant_documents(query)

# Print the retrieved documents
for doc in relevant_docs:
    print(doc.page_content)